# Team crawler notebook

## Imports

In [55]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import bs4
import re

## Crawling variables

In [2]:
headers = {
   'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
}

In [3]:
base_url = 'https://www.transfermarkt.co.uk/'

## Data

In [4]:
teams = pd.read_csv('../src/teams.csv', index_col=0)
teams_url = teams.url.values

In [5]:
print(base_url + teams_url[0][1:])

https://www.transfermarkt.co.uk/se-palmeiras-sao-paulo/startseite/verein/1023/saison_id/2017


In [6]:
s = 'https://www.transfermarkt.co.uk/se-palmeiras-sao-paulo/startseite/verein/1023/saison_id/2017' + '/plus/1'

In [7]:
requests.get(base_url + teams_url[0][1:], headers=headers)

<Response [200]>

In [123]:
url = s
print(url)
result = requests.get(url, headers=headers)
soup = BeautifulSoup(result.content, 'html5lib')
table = soup.find(name='table', attrs={'class': 'items'})
table_body = table.find(name='tbody')
rows = table_body.findAll('tr', attrs={'class': ['odd', 'even']})
t = []
for r in rows:
    t.append({
        'name': get_name(r),
        'number': get_number(r),
        'position': get_position(r),
        'date_of_birth_and_age': get_date_of_birth_and_age(r),
        'nationality': get_nationalities(r),
        'market_value': get_market_value(r)
    })

https://www.transfermarkt.co.uk/se-palmeiras-sao-paulo/startseite/verein/1023/saison_id/2017/plus/1


In [124]:
pd.DataFrame(t)

,date_of_birth_and_age,market_value,name,nationality,number,position
0,"Dec 13, 1987 (30)",£2.70m,Wéverton,Brazil,21,Goalkeeper
1,"Jul 20, 1981 (37)",£675k,Jaílson,Brazil,42,Goalkeeper
2,"Jul 9, 1978 (40)",£450k,Fernando Prass,"[Brazil, Germany]",1,Goalkeeper
3,"May 6, 1993 (25)",£3.60m,Gustavo Gómez,Paraguay,15,Centre-Back
4,"May 10, 1993 (25)",£1.80m,Luan,Brazil,13,Centre-Back
5,"Mar 7, 1993 (25)",£1.80m,Antônio Carlos,Brazil,25,Centre-Back
6,"Feb 18, 1994 (24)",£900k,Nicolás Freire,Argentina,-,Centre-Back
7,"May 18, 1981 (37)",£630k,Edú Dracena,"[Brazil, Italy]",3,Centre-Back
8,"May 3, 1997 (21)",£45k,Pedrão,Brazil,44,Centre-Back
9,"Aug 17, 1992 (26)",£3.15m,Diogo Barbosa,Brazil,6,Left-Back


In [121]:
rows[0].find('td', {'class': 'rechts hauptlink'}).contents[0]

'£2.70m\xa0'

In [122]:
def get_number(row):
    return row.find('div', {'class': 'rn_nummer'}).contents[0]


def get_name(row):
    return row.find('a', {'class': 'spielprofil_tooltip'}).contents[0]


def get_position(row):
    return row.find('table').findAll('td')[2].contents[0]


def get_date_of_birth_and_age(row):
    return row.findAll('td', {'class': 'zentriert'})[1].contents[0]


def get_nationalities(row):
    countries_img = row.findAll('img', {'class': 'flaggenrahmen'})
    nationalities = [x.get('title') for x in countries_img]
    if len(nationalities) == 1:
        return nationalities[0]
    return nationalities


def get_market_value(row):
    return row.find('td', {'class': 'rechts hauptlink'}).contents[0]

In [ ]:
leagues = pd.read_csv('../src/leagues.csv', index_col=0)

In [ ]:
teams = pd.read_csv('../src/teams.csv', index_col=0)

In [ ]:
leagues.head()

In [ ]:
teams.head()

In [ ]:
teams.shape

In [ ]:
teams.drop_duplicates()

In [ ]:
teams.merge(leagues, on='league', how='outer')